In [1]:
import os
import glob
import xml.etree.ElementTree as ET
import pandas as pd
import tensorflow as tf
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.6 MB/s eta 0:00:00


In [2]:
#mount drive
from google.colab import drive
drive.mount('/content/gdrive')

# this creates a symbolic link so that now the path /content/gdrive/My Drive/ is equal to /mydrive
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive # check if the symbolic directory works

Mounted at /content/gdrive
'Colab Notebooks'    Midterm-take-home-sources	 take-at-home.ipynb
'Embedded Systems'   MyDrive			 UAV


In [3]:
# clone tensorflow models git
!git clone https://github.com/tensorflow/models.git

# navigate to /models/research folder to compile protos
%cd models/research

# Compile protos.
!protoc object_detection/protos/*.proto --python_out=.

# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

Cloning into 'models'...
remote: Enumerating objects: 89900, done.
remote: Counting objects: 100% (3608/3608), done.
remote: Compressing objects: 100% (1935/1935), done.
remote: Total 89900 (delta 1907), reused 3278 (delta 1635), pack-reused 86292
Receiving objects: 100% (89900/89900), 606.57 MiB | 16.49 MiB/s, done.
Resolving deltas: 100% (63818/63818), done.
/content/models/research
Processing /content/models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.8 MB/s eta 0:00:00
     ━━━━━━

In [16]:
# check everything related to tensorflow models is working correctly
!python object_detection/builders/model_builder_tf2_test.py

2023-11-18 03:36:10.247190: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-18 03:36:10.247245: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-18 03:36:10.248945: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-18 03:36:12.226700: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-11-18 03:36:21.292887: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the gui

In [4]:
%cd /content/models/research/object_detection

/content/models/research/object_detection


In [5]:
# install older version of tensorflow
!pip install tensorflow[and-cuda]==2.13.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.4
    Uninstalling gast-0.5.4:
      Successfully uninstalled gast-0.5.4
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.1
    Uninstalling tensorboard-2.15.1:
      Successfully uninstalled tensorboard-2.1

Load the model and define detection function which gives bounding box informations.

In [6]:
#Loading the saved_model
import tensorflow as tf
import time
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from PIL import Image
from google.colab.patches import cv2_imshow
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

IMAGE_SIZE = (12, 12) # Output display size as you want
import matplotlib.pyplot as plt
PATH_TO_SAVED_MODEL="/mydrive/UAV/data/inference_graph/saved_model"
print('Loading model...', end='')

# Load saved model and build the detection function
detect_fn=tf.saved_model.load(PATH_TO_SAVED_MODEL)
print('Done!')

#Loading the label_map
category_index=label_map_util.create_category_index_from_labelmap("/mydrive/UAV/data/label_map.pbtxt",use_display_name=True)
#category_index=label_map_util.create_category_index_from_labelmap([path_to_label_map],use_display_name=True)

def load_image_into_numpy_array(path):

    return np.array(Image.open(path))

def get_detection(subframes_dir, imagename):
    fname = subframes_dir + imagename
    image_path = fname

    image_np = load_image_into_numpy_array(image_path)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=1,
          min_score_thresh=.07, # Adjust this value to set the minimum probability boxes to be classified as True
          agnostic_mode=False)

    return detections["detection_boxes"]

Loading model...Done!


Get bounding box informations and save them to test/detections_box folder

In [116]:
import json

# manipulates the bounding box information
def getBoxInfo(subframes_dir, imagename):
  im_width = 1280
  im_height = 720
  norm_box_boundaries = get_detection(subframes_dir, imagename) # (y_min, x_min, y_max, x_max)
  box_boundaries = [norm_box_boundaries[0,0] * im_height, norm_box_boundaries[0,1] * im_width,
                    norm_box_boundaries[0,2] * im_height, norm_box_boundaries[0,3] * im_width]
  # ((x_min, y_min, x_max, y_max))
  mapped_box_boundaries = [box_boundaries[1], box_boundaries[0], box_boundaries[3],
                          box_boundaries[2]]
  box_x = (mapped_box_boundaries[0] + mapped_box_boundaries[2]) / 2
  box_y = (mapped_box_boundaries[1] + mapped_box_boundaries[3]) / 2
  box_pos = [box_x, box_y] # (x,y)
  return box_pos, mapped_box_boundaries

# gets box data and writes to file
def writeDictToFile(subframes_dir, imagename, detections_box_dir):
  # define box information
  box_pos, box_boundaries = getBoxInfo(subframes_dir, imagename)

  # define path
  dir = detections_box_dir
  filename = imagename
  filename = filename.replace(".jpg", ".txt")
  targetDir = dir + "/" + filename

  # create dictionary
  box_info={"box_pos": box_pos, "box_boundaries": box_boundaries}

  # output data
  # print(box_info["box_pos"])
  # print(box_info["box_boundaries"])

  # write dictionary to the file
  with open(targetDir, 'w') as dict_file:
      dict_file.write(json.dumps(box_info))
      print(f"-- box info written to {i}: {filename}")

# from frame_1110 to frame_1450 in frames1, index is 3169 to 3511
subframes_dir = "/content/gdrive/MyDrive/UAV/test/frames/frames1/"
i = 0

detections_box_dir = "/mydrive/UAV/test/detections_box/detections_box_2"
# get bounding box informations for set of frames and save them to files
# which have the same names with related frames, in detections_box folder
for image in os.listdir(subframes_dir):
    imagename = os.fsdecode(image) # each frame
    if(i>=3169 and i<=3511):
      writeDictToFile(subframes_dir, imagename, detections_box_dir)
      print(f"processing {i}: {imagename}")
    elif (i>3511):
      break
    i = i + 1



-- box info written to 3169: frame_1110.txt
processing 3169: frame_1110.jpg
-- box info written to 3170: frame_1111.txt
processing 3170: frame_1111.jpg
-- box info written to 3171: frame_1112.txt
processing 3171: frame_1112.jpg
-- box info written to 3172: frame_1113.txt
processing 3172: frame_1113.jpg
-- box info written to 3173: frame_1114.txt
processing 3173: frame_1114.jpg
-- box info written to 3174: frame_1115.txt
processing 3174: frame_1115.jpg
-- box info written to 3175: frame_1116.txt
processing 3175: frame_1116.jpg
-- box info written to 3176: frame_1117.txt
processing 3176: frame_1117.jpg
-- box info written to 3177: frame_1118.txt
processing 3177: frame_1118.jpg
-- box info written to 3178: frame_1119.txt
processing 3178: frame_1119.jpg
-- box info written to 3179: frame_1120.txt
processing 3179: frame_1120.jpg
-- box info written to 3180: frame_1121.txt
processing 3180: frame_1121.jpg
-- box info written to 3181: frame_1122.txt
processing 3181: frame_1122.jpg
-- box info 

Prediction Algorithm Implementation via Kalman Filter

In [7]:
!pip install filterpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110458 sha256=9d08e44bb0c84debafaffeae351831e0a2fbbfbb72540eb0fe4cc8807f532547
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
Successfully built filterpy


In [112]:
# This library uses NumPy, SciPy, Matplotlib, and Python.
import numpy as np
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
from PIL import Image, ImageDraw
from google.colab.patches import cv2_imshow
import cv2

my_filter = KalmanFilter(dim_x=4, dim_z=2)
# position and velocity both in x and y -> dim_x=4
# sensor measurement is position in x and y -> dim_z=2

my_filter.x = np.array([1134., 281., 0., 0.])   # initial state (location and velocity)
                                                #[x,y, Vx,Vy]

my_filter.F = np.array([[1.,0.,0.05,0.],  # state transition matrix
                        [0.,1.,0.,0.05],  #[x,y,Vx,Vy]
                        [0.,0.,1.009,0.],
                        [0.,0.,0.,1.009]])


my_filter.H = np.array([[1.,0.,0.,0.],
                        [0.,1.,0.,0.]])    # Measurement function

my_filter.P *= 1000.                 # covariance matrix
my_filter.R *= 5.                      # state uncertainty
my_filter.Q = Q_discrete_white_noise(dim=4, dt=0.05, var=0.01) # process uncertainty

import json
# reads box data from file into dictionary
def readDictFromFile(detections_box_dir, imagename):
  # define path
  dir = detections_box_dir
  filename = imagename
  filename = filename.replace(".jpg", ".txt")
  sourceDir = dir + "/" + filename

  # read file
  with open(sourceDir) as file:
      data = file.read()

  # reconstructing the data as a dictionary
  box_info = json.loads(data)

  return(box_info)

def drawPrediction(box_pos_predicted, sourceImagePath, targetImagePath,
                   box_pos_his):
    image = Image.open(sourceImagePath)
    draw = ImageDraw.Draw(image)
    point_size = 3
    for box_pos in box_pos_his:
      point_pos = [box_pos[0]-point_size, box_pos[1]-point_size, box_pos[0]+point_size, box_pos[1]+point_size]
      draw.ellipse(point_pos, fill = 'yellow')
    image.save(targetImagePath)
    print(f"processed image at {targetImagePath}")
    # img = cv2.imread(targetImagePath)
    # cv2_imshow(img)

subdetections_dir = "/mydrive/UAV/test/detections/detections1"
detections_box_dir = "/mydrive/UAV/test/detections_box/detections_box_2"
subframes_dir = "/mydrive/UAV/test/frames/frames1/"
trajectory_dir = "/mydrive/UAV/test/trajectory/trajectory_2"
# i=0
box_pos_his = []
for file in os.listdir(detections_box_dir):
    filename = os.fsdecode(file) # each frame
    # read box information from file
    box_info = readDictFromFile(detections_box_dir, filename)
    box_pos = box_info["box_pos"]
    prediction = my_filter.predict()
    my_filter.update(box_pos)

   # draw the prediction
    box_pos_predicted = my_filter.x
    box_pos_his.append(box_pos_predicted)
    imagename = filename.replace(".txt", ".jpg_detected.jpg")
    detection_frame_dir = subdetections_dir + "/" + imagename
    imagename = imagename.replace(".jpg_detected.jpg", "_predicted.jpg")
    trajectory_frame_dir = trajectory_dir + "/" + imagename
    drawPrediction(box_pos_predicted, detection_frame_dir, trajectory_frame_dir,
                   box_pos_his)



processed image at /mydrive/UAV/test/trajectory/trajectory_1/frame_4250_predicted.jpg
processed image at /mydrive/UAV/test/trajectory/trajectory_1/frame_4251_predicted.jpg
processed image at /mydrive/UAV/test/trajectory/trajectory_1/frame_4253_predicted.jpg
processed image at /mydrive/UAV/test/trajectory/trajectory_1/frame_4255_predicted.jpg
processed image at /mydrive/UAV/test/trajectory/trajectory_1/frame_4254_predicted.jpg
processed image at /mydrive/UAV/test/trajectory/trajectory_1/frame_4252_predicted.jpg
processed image at /mydrive/UAV/test/trajectory/trajectory_1/frame_4256_predicted.jpg
processed image at /mydrive/UAV/test/trajectory/trajectory_1/frame_4257_predicted.jpg
processed image at /mydrive/UAV/test/trajectory/trajectory_1/frame_4258_predicted.jpg
processed image at /mydrive/UAV/test/trajectory/trajectory_1/frame_4259_predicted.jpg
processed image at /mydrive/UAV/test/trajectory/trajectory_1/frame_4260_predicted.jpg
processed image at /mydrive/UAV/test/trajectory/trajec

Create video from frames and save to folder

In [113]:
# create video from frames and save to folder
import cv2
import os

# define directories
trajectory_dir = "/content/gdrive/MyDrive/UAV/test/trajectory/trajectory_2"
video_name = 'video_trajectory_2_1110_1450.avi'
video_dir = "/content/gdrive/MyDrive/UAV/test/trajectory_video" + "/" + video_name

# create and save video to folder
images = [img for img in os.listdir(trajectory_dir) if img.endswith(".jpg")]
frame = cv2.imread(os.path.join(trajectory_dir, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_dir, 0, 20, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(trajectory_dir, image)))

cv2.destroyAllWindows()
video.release()

Output video

In [115]:
from moviepy.editor import *

# output second video
path="/content/gdrive/MyDrive/UAV/test/trajectory_video/video_trajectory_2_1110_1450.avi"

clip=VideoFileClip(path)
clip.ipython_display(width=560)

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [118]:
# is for deleting something
import shutil

shutil.rmtree("")